In [15]:
import h5py
import numpy as np
import mne
from pathlib import Path
from typing import Dict, List, Optional, Tuple
import matplotlib
#matplotlib.use("Qt5Agg")
import os

In [2]:
"""
NIX to MNE Python Converter
Complete converter for the human medial temporal lobe dataset
Based on: Boran et al. (2020) Scientific Data
"""
import h5py
import numpy as np
import mne
from pathlib import Path
from typing import Dict, List, Optional, Tuple


class NIXToMNEConverter:
    """
    Convert NIX-formatted HDF5 files to MNE-compatible formats.
    
    This converter handles the dataset from:
    "Dataset of human medial temporal lobe neurons, scalp and intracranial 
    EEG during a verbal working memory task"
    
    Features:
    - Converts iEEG and scalp EEG data
    - Preserves channel positions (MNI for iEEG, head coords for EEG)
    - Extracts trial events (Fixation, Stimulus, Maintenance, Probe, Response)
    - Creates STI channel with event codes
    - Supports single trial or full session conversion
    """
    
    def __init__(self, filepath: str, verbose: bool = True):
        """
        Initialize converter.
        
        Parameters
        ----------
        filepath : str
            Path to the NIX HDF5 file (e.g., 'Data_Subject_01_Session_01.h5')
        verbose : bool
            Print debug information
        """
        self.filepath = filepath
        self.file = None
        self.metadata = {}
        self.verbose = verbose
        
    def _log(self, message: str):
        """Print message if verbose."""
        if self.verbose:
            print(f"[NIX->MNE] {message}")
        
    def __enter__(self):
        self.file = h5py.File(self.filepath, 'r')
        self._log(f"Opened file: {self.filepath}")
        return self
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file:
            self.file.close()
    
    def _safe_decode(self, value):
        """Safely decode bytes to string."""
        if isinstance(value, bytes):
            return value.decode('utf-8')
        elif isinstance(value, np.ndarray):
            if value.dtype.kind in ('S', 'O'):
                decoded = []
                for v in value.flat:
                    if isinstance(v, bytes):
                        decoded.append(v.decode('utf-8'))
                    else:
                        decoded.append(str(v))
                return np.array(decoded).reshape(value.shape)
        return value
    
    def _get_block(self):
        """Get the data block from the file."""
        block_name = list(self.file['data'].keys())[0]
        return self.file['data'][block_name]
    
    def _extract_metadata(self) -> Dict:
        """Extract metadata from NIX file."""
        metadata = {}
        
        if 'metadata' in self.file:
            meta_group = self.file['metadata']
            
            for section_name in meta_group.keys():
                section = meta_group[section_name]
                metadata[section_name] = {}
                
                if 'properties' in section:
                    for prop_name in section['properties'].keys():
                        prop = section['properties'][prop_name]
                        try:
                            if isinstance(prop, h5py.Group) and 'values' in prop:
                                value = prop['values'][()]
                                metadata[section_name][prop_name] = self._safe_decode(value)
                        except:
                            continue
        
        self.metadata = metadata
        return metadata
    
    def _get_channel_info(self, data_type: str = 'iEEG') -> Tuple[List[str], Optional[np.ndarray]]:
        """
        Extract channel names and positions.
        
        Positions are in MNI coordinates (mm) for iEEG,
        or EEGLAB/BESA coordinates (mm) for scalp EEG.
        """
        block = self._get_block()
        
        ch_names = []
        positions = None
        
        # Get channel names from first trial data array
        group_name = 'iEEG data' if data_type == 'iEEG' else 'Scalp EEG data'
        
        if 'groups' in block and group_name in block['groups']:
            group = block['groups'][group_name]
            
            if 'data_arrays' in group:
                # Get first data array
                first_da_key = list(group['data_arrays'].keys())[0]
                first_da = group['data_arrays'][first_da_key]
                
                # Extract channel labels from dimension 1 (Set dimension)
                if 'dimensions' in first_da:
                    for dim_key in first_da['dimensions'].keys():
                        dim = first_da['dimensions'][dim_key]
                        if 'dimension_type' in dim.attrs:
                            dim_type = self._safe_decode(dim.attrs['dimension_type'])
                            if dim_type == 'set' and 'labels' in dim:
                                labels = dim['labels'][()]
                                ch_names = [self._safe_decode(label) for label in labels]
                                break
        
        # Get electrode positions - search by name attribute in data arrays
        if data_type == 'iEEG':
            elec_group_name = 'iEEG electrode information'
            coord_array_name = 'iEEG_Electrode_MNI_Coordinates'
        else:
            elec_group_name = 'Scalp EEG electrode information'
            coord_array_name = 'Scalp_Electrode_EEGLAB_BESA_Coordinates'
        
        if 'groups' in block and elec_group_name in block['groups']:
            elec_group = block['groups'][elec_group_name]
            
            if 'data_arrays' in elec_group:
                # Data arrays use UUID keys, search by name attribute
                for da_key in elec_group['data_arrays'].keys():
                    da = elec_group['data_arrays'][da_key]
                    
                    if 'name' in da.attrs:
                        da_name = self._safe_decode(da.attrs['name'])
                        
                        if da_name == coord_array_name:
                            if 'data' in da:
                                positions = da['data'][()]
                                self._log(f"Found {coord_array_name}")
                            break
        
        self._log(f"Found {len(ch_names)} channels")
        if positions is not None:
            self._log(f"Found positions: shape {positions.shape}")
        
        return ch_names, positions
    
    def _get_trial_data(self, trial_num: int, data_type: str = 'iEEG') -> Tuple[np.ndarray, float, float]:
        """
        Extract data for a specific trial.
        
        Returns
        -------
        data : ndarray
            Data array (channels x timepoints) in µV
        sfreq : float
            Sampling frequency in Hz
        offset : float
            Time offset in seconds (usually -6.0s, meaning time 0 = 6s after fixation)
        """
        block = self._get_block()
        
        group_name = 'iEEG data' if data_type == 'iEEG' else 'Scalp EEG data'
        group = block['groups'][group_name]
        
        # Find trial data array by name attribute
        trial_data_name = None
        for da_key in group['data_arrays'].keys():
            da = group['data_arrays'][da_key]
            if 'name' in da.attrs:
                da_name = self._safe_decode(da.attrs['name'])
                if f"Trial_{trial_num:02d}" in da_name:
                    trial_data_name = da_name
                    data_array = da
                    break
        
        if trial_data_name is None:
            raise ValueError(f"Trial {trial_num} not found")
        
        # Extract data
        data = data_array['data'][()]
        
        # Extract sampling parameters from dimension 2 (Sample dimension)
        sfreq = None
        offset = 0
        
        if 'dimensions' in data_array:
            for dim_key in data_array['dimensions'].keys():
                dim = data_array['dimensions'][dim_key]
                
                if 'dimension_type' in dim.attrs:
                    dim_type = self._safe_decode(dim.attrs['dimension_type'])
                    
                    if dim_type == 'sample':
                        if 'sampling_interval' in dim.attrs:
                            sampling_interval = float(dim.attrs['sampling_interval'])
                            sfreq = 1.0 / sampling_interval
                        if 'offset' in dim.attrs:
                            offset = float(dim.attrs['offset'])
                        break
        
        if sfreq is None:
            raise ValueError("Could not determine sampling frequency")
        
        return data, sfreq, offset
    
    def _get_trial_events(self, trial_num: int, data_type: str = 'iEEG') -> Dict[str, Dict]:
        """
        Extract event times for a trial.
        
        Events include: Fixation, Stimulus, Maintenance, Probe, Response
        
        Returns dict mapping event name to {'time': float, 'duration': float}
        Times are in seconds relative to trial start.
        """
        block = self._get_block()
        
        group_name = 'Trial events single tags iEEG' if data_type == 'iEEG' else 'Trial events single tags scalp EEG'
        
        events = {}
        
        if 'groups' in block and group_name in block['groups']:
            events_group = block['groups'][group_name]
            
            if 'tags' in events_group:
                trial_pattern = f"Trial_{trial_num:02d}"
                
                for tag_key in events_group['tags'].keys():
                    tag = events_group['tags'][tag_key]
                    
                    # Get event name from 'name' attribute
                    if 'name' not in tag.attrs:
                        continue
                    
                    tag_name = self._safe_decode(tag.attrs['name'])
                    
                    if trial_pattern not in tag_name:
                        continue
                    
                    # Extract event type from name
                    # Format: "Event_<EventType>_ss<setsize>_c<correct>_m<match>_Trial_<num>_iEEG"
                    parts = tag_name.split('_')
                    if len(parts) >= 2:
                        event_type = parts[1]  # e.g., "Fixation", "Stimulus"
                        
                        if 'position' in tag:
                            position = tag['position'][()]
                            event_time = float(position[1]) if len(position) >= 2 else float(position[0])
                            
                            duration = 0.0
                            if 'extent' in tag:
                                extent = tag['extent'][()]
                                duration = float(extent[1]) if len(extent) >= 2 else float(extent[0])
                            
                            events[event_type] = {
                                'time': event_time,
                                'duration': duration
                            }
        
        return events
    
    def get_trial_count(self, data_type: str = 'iEEG') -> int:
        """Get number of trials in the session."""
        block = self._get_block()
        group_name = 'iEEG data' if data_type == 'iEEG' else 'Scalp EEG data'
        
        if 'groups' in block and group_name in block['groups']:
            group = block['groups'][group_name]
            if 'data_arrays' in group:
                return len(group['data_arrays'])
        
        return 0
    
    def convert_session_to_raw(self, data_type: str = 'iEEG',
                              trials: Optional[List[int]] = None,
                              add_events: bool = True,
                              events_as_stim: bool = True) -> mne.io.RawArray:
        """
        Convert entire session to a single continuous MNE Raw object.
        
        All trials are concatenated into one continuous recording.
        
        Parameters
        ----------
        data_type : str
            'iEEG' or 'Scalp_EEG'
        trials : list, optional
            List of trial numbers (1-indexed). If None, uses all trials.
        add_events : bool
            If True, add events as annotations
        events_as_stim : bool
            If True, create a STI channel with event codes
            
        Returns
        -------
        raw : mne.io.RawArray
            Continuous Raw object with all trials concatenated
        """
        self._extract_metadata()
        
        ch_names, positions = self._get_channel_info(data_type)
        
        if not ch_names:
            raise ValueError(f"No channel names found for {data_type}")
        
        # Determine channel types
        if data_type == 'iEEG':
            ch_types = ['seeg'] * len(ch_names)
        else:
            ch_types = ['eeg'] * len(ch_names)
        
        # Add STI channel
        if add_events and events_as_stim:
            ch_names = ch_names + ['STI']
            ch_types = ch_types + ['stim']
        
        # Get trial numbers
        if trials is None:
            n_trials = self.get_trial_count(data_type)
            if n_trials > 0:
                trials = list(range(1, n_trials + 1))
            else:
                raise ValueError("No trials found")
        
        self._log(f"Concatenating {len(trials)} trials into continuous recording")
        
        # Get dimensions from first trial
        data_0, sfreq, offset = self._get_trial_data(trials[0], data_type)
        n_channels, n_times_per_trial = data_0.shape
        total_samples = n_times_per_trial * len(trials)
        
        # Allocate data array
        if add_events and events_as_stim:
            data_all = np.zeros((n_channels + 1, total_samples))
        else:
            data_all = np.zeros((n_channels, total_samples))
        
        # For annotations
        all_annotations_onset = []
        all_annotations_duration = []
        all_annotations_description = []
        event_name_to_code = {}
        next_event_code = 1
        
        # Load and concatenate all trials
        for i, trial_num in enumerate(trials):
            data, _, trial_offset = self._get_trial_data(trial_num, data_type)
            
            start_sample = i * n_times_per_trial
            end_sample = start_sample + n_times_per_trial
            
            data_all[:n_channels, start_sample:end_sample] = data
            
            # Process events
            if add_events:
                trial_events = self._get_trial_events(trial_num, data_type)
                trial_start_time = start_sample / sfreq
                
                for event_name, event_info in trial_events.items():
                    # Convert event time to absolute time in concatenated data
                    event_time_in_trial = event_info['time'] - trial_offset
                    absolute_event_time = trial_start_time + event_time_in_trial
                    
                    # Add to annotations
                    all_annotations_onset.append(absolute_event_time)
                    all_annotations_duration.append(event_info['duration'])
                    all_annotations_description.append(event_name)
                    
                    # Add to STI channel
                    if events_as_stim:
                        if event_name not in event_name_to_code:
                            event_name_to_code[event_name] = next_event_code
                            next_event_code += 1
                        
                        event_code = event_name_to_code[event_name]
                        event_start_sample = int(absolute_event_time * sfreq)
                        duration_samples = max(1, int(event_info['duration'] * sfreq))
                        event_end_sample = event_start_sample + duration_samples
                        
                        # Bounds check
                        event_start_sample = max(0, min(event_start_sample, total_samples - 1))
                        event_end_sample = max(0, min(event_end_sample, total_samples))
                        
                        if event_start_sample < event_end_sample:
                            data_all[-1, event_start_sample:event_end_sample] = event_code
        
        # Convert from µV to V (MNE standard)
        if add_events and events_as_stim:
            data_all[:-1, :] *= 1e-6
        else:
            data_all *= 1e-6
        
        # Create info
        info = mne.create_info(ch_names, sfreq, ch_types)
        
        # Add channel positions
        if positions is not None:
            try:
                from mne.io.constants import FIFF
                
                coord_frame = FIFF.FIFFV_COORD_MRI if data_type == 'iEEG' else FIFF.FIFFV_COORD_HEAD
                
                for idx in range(n_channels):
                    pos_m = positions[idx] / 1000.0  # mm to m
                    
                    if info['chs'][idx]['loc'] is None or len(info['chs'][idx]['loc']) < 12:
                        info['chs'][idx]['loc'] = np.zeros(12)
                    
                    info['chs'][idx]['loc'][0] = float(pos_m[0])
                    info['chs'][idx]['loc'][1] = float(pos_m[1])
                    info['chs'][idx]['loc'][2] = float(pos_m[2])
                    info['chs'][idx]['coord_frame'] = coord_frame
                
                self._log(f"Added positions for {n_channels} channels")
                
            except Exception as e:
                self._log(f"Warning: Could not set positions: {e}")
        
        # Create Raw object
        raw = mne.io.RawArray(data_all, info, verbose=False)
        
        raw.info['description'] = f"Session from {Path(self.filepath).name} ({len(trials)} trials)"
        
        # Store event codes
        if add_events and events_as_stim and event_name_to_code:
            raw.info['temp'] = {'event_id': event_name_to_code}
            self._log(f"Event codes: {event_name_to_code}")
        
        # Add annotations
        if add_events and all_annotations_onset:
            from mne import Annotations
            annotations = Annotations(
                onset=all_annotations_onset,
                duration=all_annotations_duration,
                description=all_annotations_description
            )
            raw.set_annotations(annotations)
            self._log(f"Added {len(all_annotations_onset)} event annotations")
        
        duration = total_samples / sfreq
        self._log(f"Created continuous Raw: {duration:.1f}s ({len(trials)} trials)")
        
        return raw
    
    def convert_session_to_raw_combined(self, 
                                       trials: Optional[List[int]] = None,
                                       add_events: bool = True,
                                       events_as_stim: bool = True) -> mne.io.RawArray:
        """
        Convert entire session with BOTH iEEG and scalp EEG in one Raw object.
        
        This creates a single Raw object containing all channels from both
        iEEG and scalp EEG recordings.
        
        Parameters
        ----------
        trials : list, optional
            List of trial numbers (1-indexed). If None, uses all trials.
        add_events : bool
            If True, add events as annotations
        events_as_stim : bool
            If True, create a STI channel with event codes
            
        Returns
        -------
        raw : mne.io.RawArray
            Continuous Raw object with both iEEG and scalp EEG channels
        """
        self._extract_metadata()
        
        # Get channel info for both types
        ieeg_ch_names, ieeg_positions = self._get_channel_info('iEEG')
        eeg_ch_names, eeg_positions = self._get_channel_info('Scalp_EEG')
        
        if not ieeg_ch_names and not eeg_ch_names:
            raise ValueError("No channels found for either iEEG or Scalp EEG")
        
        # Combine channel names and types
        all_ch_names = ieeg_ch_names + eeg_ch_names
        all_ch_types = ['seeg'] * len(ieeg_ch_names) + ['eeg'] * len(eeg_ch_names)
        
        # Add STI channel
        if add_events and events_as_stim:
            all_ch_names = all_ch_names + ['STI']
            all_ch_types = all_ch_types + ['stim']
        
        # Get trial numbers
        if trials is None:
            n_trials = self.get_trial_count('iEEG')
            if n_trials > 0:
                trials = list(range(1, n_trials + 1))
            else:
                raise ValueError("No trials found")
        
        self._log(f"Combining iEEG and scalp EEG data for {len(trials)} trials")
        
        # Get dimensions from first trial
        ieeg_data_0, ieeg_sfreq, ieeg_offset = self._get_trial_data(trials[0], 'iEEG')
        eeg_data_0, eeg_sfreq, eeg_offset = self._get_trial_data(trials[0], 'Scalp_EEG')
        
        n_ieeg_channels, n_ieeg_times = ieeg_data_0.shape
        n_eeg_channels, n_eeg_times = eeg_data_0.shape
        
        # Check if sampling rates match
        if ieeg_sfreq != eeg_sfreq:
            self._log(f"WARNING: Different sampling rates - iEEG: {ieeg_sfreq}Hz, EEG: {eeg_sfreq}Hz")
            self._log("Using iEEG sampling rate and resampling EEG data")
            # We'll need to resample EEG to match iEEG
            resample_eeg = True
            sfreq = ieeg_sfreq
            n_times_per_trial = n_ieeg_times
        else:
            resample_eeg = False
            sfreq = ieeg_sfreq
            n_times_per_trial = n_ieeg_times
        
        total_samples = n_times_per_trial * len(trials)
        total_channels = n_ieeg_channels + n_eeg_channels
        
        # Allocate data array
        if add_events and events_as_stim:
            data_all = np.zeros((total_channels + 1, total_samples))
        else:
            data_all = np.zeros((total_channels, total_samples))
        
        # For annotations
        all_annotations_onset = []
        all_annotations_duration = []
        all_annotations_description = []
        event_name_to_code = {}
        next_event_code = 1
        
        # Load and concatenate all trials
        for i, trial_num in enumerate(trials):
            # Load iEEG data
            ieeg_data, _, ieeg_trial_offset = self._get_trial_data(trial_num, 'iEEG')
            
            # Load EEG data
            eeg_data, _, eeg_trial_offset = self._get_trial_data(trial_num, 'Scalp_EEG')
            
            # Resample EEG if needed
            if resample_eeg:
                from scipy import signal
                eeg_data_resampled = signal.resample(eeg_data, n_times_per_trial, axis=1)
                eeg_data = eeg_data_resampled
            
            start_sample = i * n_times_per_trial
            end_sample = start_sample + n_times_per_trial
            
            # Combine iEEG and EEG data
            data_all[:n_ieeg_channels, start_sample:end_sample] = ieeg_data
            data_all[n_ieeg_channels:total_channels, start_sample:end_sample] = eeg_data
            
            # Process events (use iEEG events as they should match)
            if add_events:
                trial_events = self._get_trial_events(trial_num, 'iEEG')
                trial_start_time = start_sample / sfreq
                
                for event_name, event_info in trial_events.items():
                    event_time_in_trial = event_info['time'] - ieeg_trial_offset
                    absolute_event_time = trial_start_time + event_time_in_trial
                    
                    all_annotations_onset.append(absolute_event_time)
                    all_annotations_duration.append(event_info['duration'])
                    all_annotations_description.append(event_name)
                    
                    if events_as_stim:
                        if event_name not in event_name_to_code:
                            event_name_to_code[event_name] = next_event_code
                            next_event_code += 1
                        
                        event_code = event_name_to_code[event_name]
                        event_start_sample = int(absolute_event_time * sfreq)
                        duration_samples = max(1, int(event_info['duration'] * sfreq))
                        event_end_sample = event_start_sample + duration_samples
                        
                        event_start_sample = max(0, min(event_start_sample, total_samples - 1))
                        event_end_sample = max(0, min(event_end_sample, total_samples))
                        
                        if event_start_sample < event_end_sample:
                            data_all[-1, event_start_sample:event_end_sample] = event_code
        
        # Convert from µV to V
        if add_events and events_as_stim:
            data_all[:-1, :] *= 1e-6
        else:
            data_all *= 1e-6
        
        # Create info
        info = mne.create_info(all_ch_names, sfreq, all_ch_types)
        
        # Add channel positions
        if ieeg_positions is not None or eeg_positions is not None:
            try:
                from mne.io.constants import FIFF
                
                # Set iEEG positions (MNI coordinates)
                if ieeg_positions is not None:
                    for idx in range(n_ieeg_channels):
                        pos_m = ieeg_positions[idx] / 1000.0
                        
                        if info['chs'][idx]['loc'] is None or len(info['chs'][idx]['loc']) < 12:
                            info['chs'][idx]['loc'] = np.zeros(12)
                        
                        info['chs'][idx]['loc'][0] = float(pos_m[0])
                        info['chs'][idx]['loc'][1] = float(pos_m[1])
                        info['chs'][idx]['loc'][2] = float(pos_m[2])
                        info['chs'][idx]['coord_frame'] = FIFF.FIFFV_COORD_MRI
                
                # Set scalp EEG positions (head coordinates)
                if eeg_positions is not None:
                    for idx in range(n_eeg_channels):
                        pos_m = eeg_positions[idx] / 1000.0
                        ch_idx = n_ieeg_channels + idx
                        
                        if info['chs'][ch_idx]['loc'] is None or len(info['chs'][ch_idx]['loc']) < 12:
                            info['chs'][ch_idx]['loc'] = np.zeros(12)
                        
                        info['chs'][ch_idx]['loc'][0] = float(pos_m[0])
                        info['chs'][ch_idx]['loc'][1] = float(pos_m[1])
                        info['chs'][ch_idx]['loc'][2] = float(pos_m[2])
                        info['chs'][ch_idx]['coord_frame'] = FIFF.FIFFV_COORD_HEAD
                
                self._log(f"Added positions for {total_channels} channels")
                
            except Exception as e:
                self._log(f"Warning: Could not set positions: {e}")
        
        # Create Raw object
        raw = mne.io.RawArray(data_all, info, verbose=False)
        
        raw.info['description'] = f"Combined iEEG+EEG from {Path(self.filepath).name} ({len(trials)} trials)"
        
        # Store event codes
        if add_events and events_as_stim and event_name_to_code:
            raw.info['temp'] = {'event_id': event_name_to_code}
            self._log(f"Event codes: {event_name_to_code}")
        
        # Add annotations
        if add_events and all_annotations_onset:
            from mne import Annotations
            annotations = Annotations(
                onset=all_annotations_onset,
                duration=all_annotations_duration,
                description=all_annotations_description
            )
            raw.set_annotations(annotations)
            self._log(f"Added {len(all_annotations_onset)} event annotations")
        
        duration = total_samples / sfreq
        self._log(f"Created combined Raw: {duration:.1f}s, {n_ieeg_channels} iEEG + {n_eeg_channels} EEG channels")
        
        return raw
    
    def save_session_as_fif(self, output_path: str, data_type: str = 'iEEG',
                           trials: Optional[List[int]] = None,
                           add_events: bool = True,
                           events_as_stim: bool = True):
        """Save entire session as FIF file."""
        raw = self.convert_session_to_raw(data_type, trials, add_events, events_as_stim)
        raw.save(output_path, overwrite=True)
        
        duration = len(raw.times) / raw.info['sfreq']
        print(f"✓ Saved to {output_path}")
        print(f"  Duration: {duration:.1f}s, Channels: {len(raw.ch_names)}")
        
        if add_events and events_as_stim:
            event_codes = raw.info.get('temp', {}).get('event_id', {})
            print(f"  Event codes: {event_codes}")
        
        return raw


In [ ]:
# Example usage
if __name__ == "__main__":
    input_file = 'data_nix\Data_Subject_01_Session_01.h5'
    
    with NIXToMNEConverter(input_file) as converter:
        # Convert full session to continuous Raw
        raw_combined = converter.convert_session_to_raw(
            add_events=True,
            events_as_stim=True
        )
        
        # Save
        converter.save_session_as_fif('Data_converted/subject01_session01_seeg_raw.fif',
                                    data_type='Scalp_EEG'
        )
        
        # Extract events from STI channel
        events = mne.find_events(raw_combined, stim_channel='STI', consecutive=True)
        print(f"\n✓ Found {len(events)} events in STI channel")
        
        # For scalp EEG:
        # raw_eeg = converter.convert_session_to_raw(data_type='Scalp_EEG')
        # converter.save_session_as_fif('subject01_session01_eeg_raw.fif', data_type='Scalp_EEG')

directory_path = "data_nix"

for filename in os.listdir(directory_path):
    full_path = os.path.join(directory_path, filename)
    if __name__ == "__main__":
        input_file = full_path
    with NIXToMNEConverter(input_file) as converter:
        # Convert full session to continuous Raw
        raw_combined = converter.convert_session_to_raw(
            add_events=True,
            events_as_stim=True
        )
        
        # Save
        converter.save_session_as_fif(f'Data_converted/{filename}_seeg_raw.fif',
                                    data_type='Scalp_EEG'
        )
        
        # Extract events from STI channel
        events = mne.find_events(raw_combined, stim_channel='STI', consecutive=True)
        print(f"\n✓ Found {len(events)} events in STI channel")
        
        # For scalp EEG:
        # raw_eeg = converter.convert_session_to_raw(data_type='Scalp_EEG')
        # converter.save_session_as_fif('subject01_session01_eeg_raw.fif', data_type='Scalp_EEG')


In [14]:
raw = mne.io.read_raw('Data_converted\subject01_session01_seeg_raw.fif')
raw.plot()

Opening raw data file Data_converted\subject01_session01_seeg_raw.fif...
Isotrak not found
    Range : 0 ... 79999 =      0.000 ...   399.995 secs
Ready.


<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\RSKALA\AppData\Local\Temp\ipykernel_17608\3535414852.py:1: SyntaxWarning: invalid escape sequence '\s'
  raw = mne.io.read_raw('Data_converted\subject01_session01_seeg_raw.fif')


In [5]:
raw_combined.plot_sensors(ch_type="all")

<Figure size 640x640 with 1 Axes>